In [1]:
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
from upsetplot import UpSet
from upsetplot import from_indicators
import polars as pl

# from matplotlib_venn import venn2
from scipy import stats
import duckdb

sns.set_style("whitegrid")

In [2]:
with open("../ptu_derep/complete_plasmids_derep.txt") as f:
    complete_plasmids = [i.strip() for i in f]

In [3]:
with open("../ptu_derep/derep_plasmids_ids.txt") as f:
    plasmid_count = defaultdict(int)
    plasmid_oids = []
    isolate_oids = []
    derep_plasmids = [i.strip() for i in f]
    derep_plasmids = [i.split("|")[0] if "IMGPR" in i else i for i in derep_plasmids]
    for i in f:
        if i.startswith("IMGPR"):
            plasmid_oids.append(i.strip().split("_")[2])
            isolate_oids.append(i.strip().split("_")[2])
        elif i.startswith("PLSDB") or i.startswith("Refsoil"):
            plasmid_oids.append("_".join(i.strip().split("_")[1:]))
            isolate_oids.append("_".join(i.strip().split("_")[1:]))
        else:
            plasmid_oids.append(i.strip().split("|")[0])

for i in plasmid_oids:
    if i.startswith("IMGPR"):
        plasmid_count[i.split("_")[2]] += 1
    elif i.startswith("PLSDB") or i.startswith("Refsoil"):
        plasmid_count[i] += 1
    else:
        plasmid_count[i.split("|")[0]] += 1

In [4]:
total_plasmids_meta = []
total_plasmids_isolate = []
for i in derep_plasmids:
    if "PLSDB" in i or "Refsoil" in i or "IMGPR" in i:
        total_plasmids_isolate.append(i)
    else:
        total_plasmids_meta.append(i)
total_plasmids_meta = len(total_plasmids_meta)
total_plasmids_isolate = len(total_plasmids_isolate)

In [5]:
df_for_map = pd.read_csv("../env_corr/taxon_countries.tsv", sep="\t")
df_for_map["taxon_oid"] = df_for_map["taxon_oid"].astype(str)
df_for_map[df_for_map["taxon_oid"].isin(plasmid_oids + isolate_oids)]

df_for_map.head()

,taxon_oid,Ecosystem Subtype,Latitude,Longitude,Origin,Isolation Country,Ecosystem Subtype Custom,Plasmid Count,soil_class,bdod (cg/cm³),...,silt (g/kg),soc (dg/kg),geometry,index_right,ECO_NAME,WWF_REALM,RealmMHT,WWF_REALM2,WWF_MHTNUM,WWF_MHTNAM
0,3300049023,Grasslands,38.53,-121.78,Meta,USA,Grasslands,36,Luvisols,152.0,...,562.0,335.0,POINT (-121.78 38.53),761.0,Great Central Valley,NaN,NA12,Nearctic,12.0,"Mediterranean Forests, Woodlands and Scrub"
1,3300012840,Grasslands,43.07,-89.40,Meta,USA,Grasslands,21,Luvisols,NaN,...,NaN,NaN,POINT (-89.4 43.07),187.0,Prairie-Forest Border,NaN,NA4,Nearctic,4.0,Temperate Broadleaf and Mixed Forests
2,3300039503,Unclassified,63.88,-149.23,Meta,USA,Unclassified,3,Cambisols,60.0,...,496.0,2409.0,POINT (-149.23 63.88),734.0,Alaska Range,NaN,NA6,Nearctic,6.0,Boreal Forests/Taiga
3,3300042005,Rhizosphere,41.20,-97.94,Meta,USA,Rhizosphere,6,Kastanozems,135.0,...,242.0,301.0,POINT (-97.94 41.2),747.0,Central Mixed-Grass Prairie,NaN,NA8,Nearctic,8.0,"Temperate Grasslands, Savannas and Shrublands"
4,3300049265,Agricultural land,38.55,-121.87,Meta,USA,Agricultural land,1,Vertisols,158.0,...,508.0,274.0,POINT (-121.87 38.55),761.0,Great Central Valley,NaN,NA12,Nearctic,12.0,"Mediterranean Forests, Woodlands and Scrub"


In [6]:
with duckdb.connect("../soil_plasmid.db") as con:
    pfams_hits = (
        con.sql("SELECT * FROM hmm_outputs WHERE Query_name LIKE 'PF%'").pl().lazy()
    )
    ptus = con.sql("SELECT * FROM ptu_derep").df()

In [7]:
pfams_hits = (
    pfams_hits.with_columns(
        pl.col("Hit_name")
        .map_elements(
            lambda x: "Isolate"
            if any([i for i in ["IMGPR", "PLSDB", "Refsoil"] if i in x])
            else "Meta"
        )
        .alias("Origin"),
    )
    .with_columns(
        pl.col("Hit_name")
        .map_elements(
            lambda x: (
                "_".join(x.split("_")[0:-1]) if "IMGPR" not in x else x.split("|")[0]
            )
        )
        .alias("Plasmid_name")
    )
    .filter(pl.col("Plasmid_name").is_in(derep_plasmids))
    .with_columns(
        pl.col("Plasmid_name")
        .map_elements(
            lambda x: x.split("|")[0] if "IMGPR" not in x else x.split("_")[2]
        )
        .alias("taxon_oid")
    )
    .collect()
)

In [8]:
hit_descriptions = pd.read_csv(
    "../helper_tables/PFAM_NCBIFAM_KOFAM_entries.tsv", sep="\t", index_col=0
)

pfam_descriptions = hit_descriptions[hit_descriptions["Source Database"] == "pfam"]

kofam_descriptions = hit_descriptions[hit_descriptions["Source Database"] == "kofam"]
pfam_descriptions.head()

,Name,Source Database,Type,Integrated Into,Integrated Signatures,GO Terms
Accession,,,,,,
PF00001,7 transmembrane receptor (rhodopsin family),pfam,family,IPR000276,NaN,NaN
PF00002,7 transmembrane receptor (Secretin family),pfam,family,IPR000832,NaN,NaN
PF00003,7 transmembrane sweet-taste receptor of 3 GCPR,pfam,domain,IPR017978,NaN,NaN
PF00004,ATPase family associated with various cellular...,pfam,domain,IPR003959,NaN,NaN
PF00005,ABC transporter,pfam,domain,IPR003439,NaN,NaN


In [9]:
ptus_mapping = {}

for idx, row in ptus.iterrows():
    for plasmid in row[2].split(","):
        plasmid = plasmid.split("|")[0] if "IMGPR" in plasmid else plasmid
        ptus_mapping[plasmid] = idx

/tmp/ipykernel_3188244/600427213.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for plasmid in row[2].split(","):


In [10]:
pfams_hits = (
    pfams_hits.join(pl.from_pandas(df_for_map), on="taxon_oid")
    .with_columns(
        [
            pl.col("Plasmid_name")
            .map_elements(lambda x: ptus_mapping.get(x))
            .alias("PTU"),
            pl.col("Query_name")
            .map_elements(lambda x: pfam_descriptions.loc[x.split(".")[0], "Name"])
            .alias("Description"),
            pl.col("Query_name")
            .map_elements(lambda x: x.split(".")[0])
            .alias("Query_clean"),
            pl.col("Plasmid_name").is_in(complete_plasmids).alias("complete"),
            pl.col("phh2o (pH*10)")
            .map_elements(
                lambda x: "Acid" if x <= 60 else "Basic" if x >= 80 else "Neutral"
            )
            .alias("ph_class"),
        ]
    )
    .drop(
        [
            "PTU Count",
            "PTU Count Bins",
            "Plasmid Count Bins",
            "Plasmid Count",
            "Origin_right",
        ]
    )
    .filter(pl.col("PTU").is_not_null())
)

pfams_hits.head()

Query_name,Hit_name,Hit_evalue,Hit_score,HMM_coverage,Origin,Plasmid_name,taxon_oid,Ecosystem Subtype,Latitude,Longitude,Isolation Country,Ecosystem Subtype Custom,soil_class,bdod (cg/cm³),cec (mmol(c)/kg),cfvo (cm³/dm³),clay (g/kg),nitrogen (cg/kg),ocd (dg/dm³),phh2o (pH*10),sand (g/kg),silt (g/kg),soc (dg/kg),geometry,index_right,ECO_NAME,WWF_REALM,RealmMHT,WWF_REALM2,WWF_MHTNUM,WWF_MHTNAM,PTU,Description,Query_clean,complete,ph_class
str,str,f64,f64,f64,str,str,str,str,f64,f64,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,str,i64,str,str,bool,str
"""PF09299.15""","""3300025910|Ga0…",9.1000e-18,63.19,0.98,"""Meta""","""3300025910|Ga0…","""3300025910""","""Rhizosphere""",42.39,-85.37,"""USA""","""Rhizosphere""","""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…",10972,"""Mu transposase…","""PF09299""",false,"""Acid"""
"""PF13379.10""","""3300025911|Ga0…",8.4900e-17,60.94,0.89,"""Meta""","""3300025911|Ga0…","""3300025911""","""Rhizosphere""",42.39,-85.37,"""USA""","""Rhizosphere""","""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…",14106,"""NMT1-like fami…","""PF13379""",false,"""Acid"""
"""PF00345.24""","""3300025909|Ga0…",8.5900e-18,63.64,0.98,"""Meta""","""3300025909|Ga0…","""3300025909""","""Rhizosphere""",42.39,-85.37,"""USA""","""Rhizosphere""","""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…",4123,"""Pili and flage…","""PF00345""",false,"""Acid"""
"""PF02195.22""","""3300025912|Ga0…",4.5400e-25,86.78,0.98,"""Meta""","""3300025912|Ga0…","""3300025912""","""Rhizosphere""",42.39,-85.37,"""USA""","""Rhizosphere""","""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…",6070,"""ParB/Sulfiredo…","""PF02195""",false,"""Acid"""
"""PF00989.29""","""3300025911|Ga0…",3.4300e-29,100.3,0.96,"""Meta""","""3300025911|Ga0…","""3300025911""","""Rhizosphere""",42.39,-85.37,"""USA""","""Rhizosphere""","""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…",2832,"""PAS fold""","""PF00989""",false,"""Acid"""


---
## General PFam enrichment

Isso pq os plasmídeos de isolado tendem a ser maiores, então a chance de eles terem um Pfam X é maior. Oq a gente quer saber é se a chance de um gene aleatório de um plasmídeo meta ter o Pfam X é maior do que a chance de um gene aleatório de plasmídeo isolado

Eu faria uma outra modificação, eu calcularia isso por plasmídeo. Pra cada plasmídeo vc vai calcular: Número de genes com Pfam X / número de genes total

assim, vc fica com um número entre 0 a 1 pra cada plasmídeo, pra cada pfam

depois, pra cada pfam, vc pode fazer um teste T pra ver se tem diferença das médias

Fazer para cada PF 0 e 1 de plasmídeo

In [11]:
test = (
    (
        pfams_hits.unique(subset=["Query_clean", "Plasmid_name"])
        .group_by(["Query_clean", "Origin"])
        .agg(pl.col("Plasmid_name").n_unique().alias("n_plasmids"))
    )
    .pivot(index="Query_clean", columns="Origin", values="n_plasmids")
    .fill_null(0)
    .with_columns(
        [
            pl.col("Meta")
            .map_elements(lambda x: total_plasmids_meta - x)
            .alias("Meta_no"),
            pl.col("Isolate")
            .map_elements(lambda x: total_plasmids_isolate - x)
            .alias("Isolate_no"),
        ]
    )
)

In [12]:
test.filter(pl.col("Query_clean") == "PF08017")

Query_clean,Isolate,Meta,Meta_no,Isolate_no
str,u32,u32,i64,i64
"""PF08017""",1,0,90200,8080


In [13]:
def calc_odds_ratio(element_type):
    contingency_df = test.filter(pl.col("Query_clean").eq(element_type))
    meta_with_element = contingency_df.get_column("Meta")[0]
    isolate_with_element = contingency_df.get_column("Isolate")[0]

    meta_without_element = contingency_df.get_column("Meta_no")[0]
    isolate_without_element = contingency_df.get_column("Isolate_no")[0]

    # without pseudo counts for fisher exact
    table_fisher = np.array(
        [
            [meta_with_element, isolate_with_element],
            [
                meta_without_element,
                isolate_without_element,
            ],
        ]
    )

    fisher_exact_p = stats.fisher_exact(table_fisher).pvalue
    res = stats.contingency.odds_ratio(table_fisher + 1)

    return (
        float(table_fisher[0][0]),
        float(table_fisher[0][1]),
        np.log(res.statistic),
        fisher_exact_p,
    )

In [14]:
a = (
    pfams_hits.select(pl.col("Query_clean").unique())
    .with_columns(results=pl.col("Query_clean").map_elements(calc_odds_ratio))
    .with_columns(pl.col("results").list.to_struct())
    .unnest("results")
    .rename(
        {"field_0": "Meta", "field_1": "Isolate", "field_2": "log_odds", "field_3": "p"}
    )
)

a.head()

Query_clean,Meta,Isolate,log_odds,p
str,f64,f64,f64,f64
"""PF17954""",82.0,85.0,-2.457513,4.5753e-47
"""PF09615""",13.0,3.0,-1.159846,0.139318
"""PF13750""",19.0,20.0,-2.4634,2.8762e-12
"""PF12399""",324.0,508.0,-2.922269,0.0
"""PF00460""",57.0,46.0,-2.207132,4.1928e-23


In [15]:
b = a.filter(pl.col("Meta") + pl.col("Isolate") >= 100)
p_adjusted = stats.false_discovery_control(b["p"])
b = b.with_columns(padj=p_adjusted)
b.head()

Query_clean,Meta,Isolate,log_odds,p,padj
str,f64,f64,f64,f64,f64
"""PF17954""",82.0,85.0,-2.457513,4.5753e-47,7.0527e-47
"""PF12399""",324.0,508.0,-2.922269,0.0,0.0
"""PF00460""",57.0,46.0,-2.207132,4.1928e-23,5.0967e-23
"""PF20398""",61.0,99.0,-2.902013,1.3158e-65,2.4810e-65
"""PF01757""",429.0,484.0,-2.589689,2.7568e-274,1.9465e-273


In [16]:
#b.write_csv("pfam_odds.tsv", separator="\t")

---

### Kofam analysis

In [27]:
with duckdb.connect("../soil_plasmid.db") as con:
    kofam_hits = (
        con.sql("SELECT * FROM hmm_outputs WHERE Query_name LIKE 'K%'").pl().lazy()
    )

In [28]:
# 2min 25s computation
kofam_hits = (
    kofam_hits.with_columns(
        [
            pl.col("Hit_name")
            .map_elements(
                lambda x: (
                    "Isolate"
                    if any(i for i in ["IMGPR", "PLSDB", "Refsoil"] if i in x)
                    else "Meta"
                )
            )
            .alias("Origin"),
            pl.col("Hit_name")
            .map_elements(
                lambda x: (
                    "_".join(x.split("_")[0:-1])
                    if "IMGPR" not in x
                    else x.split("|")[0]
                )
            )
            .alias("Plasmid_name"),
        ]
    )
    .filter(pl.col("Plasmid_name").is_in(derep_plasmids))
    .with_columns(
        [
            pl.col("Plasmid_name")
            .map_elements(
                lambda x: x.split("|")[0] if "IMGPR" not in x else x.split("_")[2]
            )
            .alias("taxon_oid"),
            pl.col("Plasmid_name").is_in(complete_plasmids).alias("complete"),
        ]
    )
    .with_columns(
        pl.col("Plasmid_name")
        .map_elements(lambda x: ptus_mapping.get(x, np.nan))
        .alias("PTU"),
    )
    .filter(pl.col("PTU").is_not_null())
    .join(pl.from_pandas(df_for_map).lazy(), on="taxon_oid")
    .collect()
)

kofam_hits.head()

Query_name,Hit_name,Hit_evalue,Hit_score,HMM_coverage,Origin,Plasmid_name,taxon_oid,complete,PTU,Ecosystem Subtype,Latitude,Longitude,Origin_right,Isolation Country,Ecosystem Subtype Custom,Plasmid Count,soil_class,bdod (cg/cm³),cec (mmol(c)/kg),cfvo (cm³/dm³),clay (g/kg),nitrogen (cg/kg),ocd (dg/dm³),phh2o (pH*10),sand (g/kg),silt (g/kg),soc (dg/kg),geometry,index_right,ECO_NAME,WWF_REALM,RealmMHT,WWF_REALM2,WWF_MHTNUM,WWF_MHTNAM
str,str,f64,f64,f64,str,str,str,bool,i64,str,f64,f64,str,str,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,str
"""K05847""","""3300025728|Ga0…",4.0800e-90,302.13,0.73,"""Meta""","""3300025728|Ga0…","""3300025728""",false,12992,"""Rhizosphere""",42.39,-85.37,"""Meta""","""USA""","""Rhizosphere""",146,"""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…"
"""K05847""","""3300025728|Ga0…",8.0900e-41,139.81,0.67,"""Meta""","""3300025728|Ga0…","""3300025728""",false,3288,"""Rhizosphere""",42.39,-85.37,"""Meta""","""USA""","""Rhizosphere""",146,"""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…"
"""K05874""","""3300025735|Ga0…",1.7900e-144,482.27,0.83,"""Meta""","""3300025735|Ga0…","""3300025735""",true,73,"""Rhizosphere""",42.39,-85.37,"""Meta""","""USA""","""Rhizosphere""",168,"""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…"
"""K05877""","""3300025728|Ga0…",1.9500e-118,395.79,0.95,"""Meta""","""3300025728|Ga0…","""3300025728""",false,12058,"""Rhizosphere""",42.39,-85.37,"""Meta""","""USA""","""Rhizosphere""",146,"""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…"
"""K05877""","""3300025735|Ga0…",1.4400e-55,188.32,0.76,"""Meta""","""3300025735|Ga0…","""3300025735""",false,29571,"""Rhizosphere""",42.39,-85.37,"""Meta""","""USA""","""Rhizosphere""",168,"""Luvisols""",135.0,157.0,73.0,167.0,897.0,394.0,58.0,600.0,233.0,606.0,"""POINT (-85.37 …",182.0,"""North Central …",null,"""NA4""","""Nearctic""",4.0,"""Temperate Broa…"


KO enrichment

In [29]:
test = (
    (
        kofam_hits.unique(subset=["Query_name", "Plasmid_name"])
        .group_by(["Query_name", "Origin"])
        .agg(pl.col("Plasmid_name").n_unique().alias("n_plasmids"))
    )
    .pivot(index="Query_name", columns="Origin", values="n_plasmids")
    .fill_null(0)
    .with_columns(
        [
            pl.col("Meta")
            .map_elements(lambda x: total_plasmids_meta - x)
            .alias("Meta_no"),
            pl.col("Isolate")
            .map_elements(lambda x: total_plasmids_isolate - x)
            .alias("Isolate_no"),
        ]
    )
)

In [30]:
def calc_odds_ratio(element_type):
    contingency_df = test.filter(pl.col("Query_name").eq(element_type))
    meta_with_element = contingency_df.get_column("Meta")[0]
    isolate_with_element = contingency_df.get_column("Isolate")[0]

    meta_without_element = contingency_df.get_column("Meta_no")[0]
    isolate_without_element = contingency_df.get_column("Isolate_no")[0]

    # without pseudo counts for fisher exact
    table_fisher = np.array(
        [
            [meta_with_element, isolate_with_element],
            [
                meta_without_element,
                isolate_without_element,
            ],
        ]
    )

    fisher_exact_p = stats.fisher_exact(table_fisher).pvalue
    res = stats.contingency.odds_ratio(table_fisher + 1)

    return (
        float(table_fisher[0][0]),
        float(table_fisher[0][1]),
        np.log(res.statistic),
        fisher_exact_p,
    )

In [31]:
a = (
    kofam_hits.select(pl.col("Query_name").unique())
    .with_columns(results=pl.col("Query_name").map_elements(calc_odds_ratio))
    .with_columns(pl.col("results").list.to_struct())
    .unnest("results")
    .rename(
        {"field_0": "Meta", "field_1": "Isolate", "field_2": "log_odds", "field_3": "p"}
    )
)

a.head()

Query_name,Meta,Isolate,log_odds,p
str,f64,f64,f64,f64
"""K25522""",5.0,20.0,-3.667492,6.9070e-18
"""K19555""",4.0,8.0,-3.001058,7.6150e-7
"""K02341""",53.0,62.0,-2.573603,1.1123e-36
"""K01964""",171.0,297.0,-2.997467,2.7079e-199
"""K04015""",17.0,19.0,-2.519867,5.1615e-12


In [32]:
b = a.filter(pl.col("Meta") + pl.col("Isolate") >= 100)
p_adjusted = stats.false_discovery_control(b["p"])
b = b.with_columns(padj=p_adjusted)
b.head()

Query_name,Meta,Isolate,log_odds,p,padj
str,f64,f64,f64,f64,f64
"""K02341""",53.0,62.0,-2.573603,1.1123e-36,1.2541e-36
"""K01964""",171.0,297.0,-2.997467,2.7079e-199,6.9830e-199
"""K19577""",3528.0,1978.0,-2.074733,0.0,0.0
"""K05877""",588.0,443.0,-2.179584,9.2113e-203,2.4139e-202
"""K22431""",126.0,206.0,-2.925281,1.7652e-135,3.5966e-135


In [33]:
b.write_csv("kofam_odds.tsv", separator="\t")

NCBIFams

In [34]:
with duckdb.connect("../soil_plasmid.db") as con:
    ncbifam_hits = (
        con.sql(
            "SELECT * FROM hmm_outputs WHERE Query_name LIKE 'N%' OR Query_name LIKE 'TIGR%'"
        )
        .pl()
        .lazy()
    )

ncbifam_hits = (
    ncbifam_hits.with_columns(
        [
            pl.col("Hit_name")
            .map_elements(
                lambda x: (
                    "Isolate"
                    if any(i for i in ["IMGPR", "PLSDB", "Refsoil"] if i in x)
                    else "Meta"
                )
            )
            .alias("Origin"),
            pl.col("Hit_name")
            .map_elements(
                lambda x: (
                    "_".join(x.split("_")[0:-1])
                    if "IMGPR" not in x
                    else x.split("|")[0]
                )
            )
            .alias("Plasmid_name"),
        ]
    )
    .filter(pl.col("Plasmid_name").is_in(derep_plasmids))
    .with_columns(
        [
            pl.col("Plasmid_name")
            .map_elements(
                lambda x: x.split("|")[0] if "IMGPR" not in x else x.split("_")[2]
            )
            .alias("taxon_oid"),
            pl.col("Plasmid_name").is_in(complete_plasmids).alias("complete"),
        ]
    )
    .with_columns(
        pl.col("Plasmid_name")
        .map_elements(lambda x: ptus_mapping.get(x, np.nan))
        .alias("PTU"),
    )
    .filter(pl.col("PTU").is_not_null())
    .join(pl.from_pandas(df_for_map).lazy(), on="taxon_oid")
    .collect()
)

ncbifam_hits.head()

Query_name,Hit_name,Hit_evalue,Hit_score,HMM_coverage,Origin,Plasmid_name,taxon_oid,complete,PTU,Ecosystem Subtype,Latitude,Longitude,Origin_right,Isolation Country,Ecosystem Subtype Custom,Plasmid Count,soil_class,bdod (cg/cm³),cec (mmol(c)/kg),cfvo (cm³/dm³),clay (g/kg),nitrogen (cg/kg),ocd (dg/dm³),phh2o (pH*10),sand (g/kg),silt (g/kg),soc (dg/kg),geometry,index_right,ECO_NAME,WWF_REALM,RealmMHT,WWF_REALM2,WWF_MHTNUM,WWF_MHTNAM
str,str,f64,f64,f64,str,str,str,bool,i64,str,f64,f64,str,str,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,str,str,f64,str
"""TIGR03446.1""","""3300046782|Ga0…",6.7500e-123,414.68,0.67,"""Meta""","""3300046782|Ga0…","""3300046782""",false,1257,"""Agricultural l…",46.12,-123.27,"""Meta""","""USA""","""Agricultural l…",45,"""Cambisols""",111.0,281.0,101.0,273.0,1124.0,582.0,53.0,279.0,448.0,852.0,"""POINT (-123.27…",723.0,"""Pacific Northw…",null,"""NA5""","""Nearctic""",5.0,"""Temperate Coni…"
"""TIGR04498.1""","""3300046780|Ga0…",1.2700e-17,69.34,0.98,"""Meta""","""3300046780|Ga0…","""3300046780""",false,239,"""Agricultural l…",46.12,-123.27,"""Meta""","""USA""","""Agricultural l…",25,"""Cambisols""",111.0,281.0,101.0,273.0,1124.0,582.0,53.0,279.0,448.0,852.0,"""POINT (-123.27…",723.0,"""Pacific Northw…",null,"""NA5""","""Nearctic""",5.0,"""Temperate Coni…"
"""NF033516.0""","""3300041809|Ga0…",1.5200e-66,230.06,0.72,"""Meta""","""3300041809|Ga0…","""3300041809""",false,18944,"""Grasslands""",38.98,-106.97,"""Meta""","""USA""","""Grasslands""",924,"""Cambisols""",111.0,319.0,280.0,164.0,708.0,439.0,55.0,498.0,337.0,1041.0,"""POINT (-106.97…",727.0,"""Southern Rocky…",null,"""NA5""","""Nearctic""",5.0,"""Temperate Coni…"
"""NF033542.1""","""3300041809|Ga0…",1.6000e-90,308.64,0.86,"""Meta""","""3300041809|Ga0…","""3300041809""",false,18610,"""Grasslands""",38.98,-106.97,"""Meta""","""USA""","""Grasslands""",924,"""Cambisols""",111.0,319.0,280.0,164.0,708.0,439.0,55.0,498.0,337.0,1041.0,"""POINT (-106.97…",727.0,"""Southern Rocky…",null,"""NA5""","""Nearctic""",5.0,"""Temperate Coni…"
"""NF041492.1""","""3300041965|Ga0…",1.9600e-74,255.81,1.0,"""Meta""","""3300041965|Ga0…","""3300041965""",false,19396,"""Grasslands""",46.25,-119.73,"""Meta""","""USA""","""Grasslands""",574,"""Cambisols""",127.0,151.0,33.0,75.0,524.0,223.0,75.0,507.0,419.0,426.0,"""POINT (-119.73…",766.0,"""Columbia Plate…",null,"""NA13""","""Nearctic""",13.0,"""Deserts and Xe…"


In [35]:
test = (
    (
        ncbifam_hits.unique(subset=["Query_name", "Plasmid_name"])
        .group_by(["Query_name", "Origin"])
        .agg(pl.col("Plasmid_name").n_unique().alias("n_plasmids"))
    )
    .pivot(index="Query_name", columns="Origin", values="n_plasmids")
    .fill_null(0)
    .with_columns(
        [
            pl.col("Meta")
            .map_elements(lambda x: total_plasmids_meta - x)
            .alias("Meta_no"),
            pl.col("Isolate")
            .map_elements(lambda x: total_plasmids_isolate - x)
            .alias("Isolate_no"),
        ]
    )
)

In [36]:
def calc_odds_ratio(element_type):
    contingency_df = test.filter(pl.col("Query_name").eq(element_type))
    meta_with_element = contingency_df.get_column("Meta")[0]
    isolate_with_element = contingency_df.get_column("Isolate")[0]

    meta_without_element = contingency_df.get_column("Meta_no")[0]
    isolate_without_element = contingency_df.get_column("Isolate_no")[0]

    # without pseudo counts for fisher exact
    table_fisher = np.array(
        [
            [meta_with_element, isolate_with_element],
            [
                meta_without_element,
                isolate_without_element,
            ],
        ]
    )

    fisher_exact_p = stats.fisher_exact(table_fisher).pvalue
    res = stats.contingency.odds_ratio(table_fisher + 1)

    return (
        float(table_fisher[0][0]),
        float(table_fisher[0][1]),
        np.log(res.statistic),
        fisher_exact_p,
    )

In [37]:
a = (
    ncbifam_hits.select(pl.col("Query_name").unique())
    .with_columns(results=pl.col("Query_name").map_elements(calc_odds_ratio))
    .with_columns(pl.col("results").list.to_struct())
    .unnest("results")
    .rename(
        {"field_0": "Meta", "field_1": "Isolate", "field_2": "log_odds", "field_3": "p"}
    )
)

a.head()

Query_name,Meta,Isolate,log_odds,p
str,f64,f64,f64,f64
"""NF008576.0""",7.0,0.0,-0.332878,1.0
"""NF001304.0""",2.0,0.0,-1.313744,1.0
"""NF004622.0""",3.0,0.0,-1.026058,1.0
"""NF004936.0""",3.0,2.0,-2.124887,0.057155
"""NF008606.0""",6.0,14.0,-3.176127,1.5357e-11


In [38]:
b = a.filter(pl.col("Meta") + pl.col("Isolate") >= 100)
p_adjusted = stats.false_discovery_control(b["p"])
b = b.with_columns(padj=p_adjusted)
b.head()

Query_name,Meta,Isolate,log_odds,p,padj
str,f64,f64,f64,f64,f64
"""TIGR02666.1""",78.0,36.0,-1.657438,7.6557e-13,9.3089e-13
"""NF010407.0""",107.0,125.0,-2.58089,2.2339e-72,7.6188e-72
"""NF010398.0""",113.0,114.0,-2.43403,8.5046e-62,2.3549e-61
"""NF001923.0""",80.0,96.0,-2.603662,1.3919e-56,3.4965e-56
"""TIGR02315.1""",107.0,229.0,-3.195824,2.2016e-164,2.6387e-163


In [39]:
b.write_csv("ncbifam_odds.tsv", separator="\t")

Odds for functions in plasmids targeted by crisprs in hosts

In [45]:
total_plasmids = total_plasmids_isolate + total_plasmids_meta

In [48]:
with open('../crispr_search/meta_spacers/host_crispr_targeted_plasmids.txt') as f:
    crispr_targeted_plasmids = [i.strip() for i in f]
pfams_hits = pfams_hits.with_columns(crispr_targeted=pl.when(pl.col('Plasmid_name').is_in(crispr_targeted_plasmids)).then(pl.lit('targeted')).otherwise(pl.lit('not_targeted')))

test = (
    (
        pfams_hits.unique(subset=["Query_clean", "Plasmid_name"])
        .group_by(["Query_clean", "crispr_targeted"])
        .agg(pl.col("Plasmid_name").n_unique().alias("n_plasmids"))
    )
    .pivot(index="Query_clean", columns="crispr_targeted", values="n_plasmids")
    .fill_null(0)
    .with_columns(
        [
            pl.col("targeted")
            .map_elements(lambda x: len(crispr_targeted_plasmids) - x)
            .alias("targeted_no"),
            pl.col("not_targeted")
            .map_elements(lambda x: (total_plasmids - len(crispr_targeted_plasmids)) - x)
            .alias("not_targeted_no"),
        ]
    )
)

In [49]:
def calc_odds_ratio(element_type):
    contingency_df = test.filter(pl.col("Query_clean").eq(element_type))
    crispr_targeted_with_element = contingency_df.get_column("targeted")[0]
    not_crispr_targeted_with_element = contingency_df.get_column("not_targeted")[0]

    crispr_targeted_without_element = contingency_df.get_column("targeted_no")[0]
    not_crispr_targeted_without_element = contingency_df.get_column("not_targeted_no")[0]

    # without pseudo counts for fisher exact
    table_fisher = np.array(
        [
            [crispr_targeted_with_element, not_crispr_targeted_with_element],
            [
                crispr_targeted_without_element,
                not_crispr_targeted_without_element,
            ],
        ]
    )

    fisher_exact_p = stats.fisher_exact(table_fisher).pvalue
    res = stats.contingency.odds_ratio(table_fisher + 1)

    return (
        float(table_fisher[0][0]),
        float(table_fisher[0][1]),
        np.log(res.statistic),
        fisher_exact_p,
    )

In [52]:
a = (
    pfams_hits.select(pl.col("Query_clean").unique())
    .with_columns(results=pl.col("Query_clean").map_elements(calc_odds_ratio))
    .with_columns(pl.col("results").list.to_struct())
    .unnest("results")
    .rename(
        {"field_0": "targeted", "field_1": "not_targeted", "field_2": "log_odds", "field_3": "p"}
    )
)

a.head()

Query_clean,targeted,not_targeted,log_odds,p
str,f64,f64,f64,f64
"""PF07453""",0.0,9.0,3.094646,1.0
"""PF06796""",0.0,45.0,1.56838,1.0
"""PF19460""",0.0,4.0,3.787673,1.0
"""PF13351""",0.0,115.0,0.642744,1.0
"""PF01816""",1.0,53.0,2.10333,0.216096


In [67]:
b = a.filter(pl.col("targeted") + pl.col("not_targeted") >= 100)
p_adjusted = stats.false_discovery_control(b["p"])
b = b.with_columns(padj=p_adjusted)
b.head()

Query_clean,targeted,not_targeted,log_odds,p,padj
str,f64,f64,f64,f64,f64
"""PF13351""",0.0,115.0,0.642744,1.0,1.0
"""PF12728""",34.0,7634.0,0.010986,1.0,1.0
"""PF00474""",1.0,379.0,0.148872,1.0,1.0
"""PF11198""",3.0,423.0,0.736531,0.446428,0.864931
"""PF17802""",0.0,272.0,-0.214735,0.639014,0.977832


In [69]:
b.filter((pl.col("padj") < 0.05)).sort("log_odds", descending=True)

Query_clean,targeted,not_targeted,log_odds,p,padj
str,f64,f64,f64,f64,f64
"""PF06952""",54.0,130.0,4.657631,5.6723e-82,1.3177e-78
"""PF09827""",41.0,98.0,4.635501,1.8087e-62,1.4005e-59
"""PF06290""",45.0,117.0,4.560753,3.9078e-67,4.5390e-64
"""PF05818""",28.0,101.0,4.203602,9.2132e-39,2.3780e-36
"""PF10723""",28.0,102.0,4.19384,1.1694e-38,2.7166e-36
"""PF07057""",35.0,151.0,4.037485,8.4197e-46,3.2598e-43
"""PF06006""",27.0,118.0,4.011851,1.8540e-35,3.5891e-33
"""PF09707""",19.0,83.0,4.004993,2.4573e-25,2.0387e-23
"""PF01867""",41.0,190.0,3.977684,2.2728e-52,1.3199e-49
